In this lab we are going to implement different aggregating functions to create differente versions of pooling layers. Afterwards, the analysis of the results derived from different models will be performed, as well as the identification of different problems that will have appeared in the process. 

# Lab 1: Modifications of Pooling Layers

## Preparing the environment

The libraries that will be needed shall be imported:

In [27]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # FFFFF

# Data loading
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Auxiliary functions
from torch.utils.tensorboard import SummaryWriter  # Used for Tensorboard logging
import os
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
import datetime

# Math
import math

Firstly, we are going to establish a working hierarchy to have everything correctly organised:

    ├── data               <- Directory for storing datasets
    └── reports            <- Outputs produced by the model
        ├── models         <- Trained and serialized models
        ├── results        <- Results obtained after training, testing on the test set
        └── runs           <- Logs generated during training, interpretable by tensorboard (tensorboard --logdir [namedir])

Consequently, we establish the following paths that shall be used later

In [3]:

PATH_ROOT = os.path.join('.')
# Path for data:
PATH_DATA = os.path.join(PATH_ROOT, 'data')
# Path for models:
PATH_MODELS = os.path.join(PATH_ROOT, 'reports', 'models')
# Path for results:
PATH_RESULTS = os.path.join(PATH_ROOT, 'reports', 'results')
# Path for runnings:
PATH_RUNS = os.path.join(PATH_ROOT, 'reports', 'runs')

# For each session we create a new folder from the datetime of its execution. 
date = datetime.datetime.now()
test_name = str(date.year) + '_' + str(date.month) + '_' +  str(date.day) + '__' + str(date.hour) + '_' + str(date.minute)
print('Proofs Folder Path: {}'.format(test_name))
models_folder = os.path.join(PATH_MODELS, test_name)
try:
    os.mkdir(models_folder)
except:
    print(f'Folder {models_folder} already existed.')
results_folder = os.path.join(PATH_RESULTS, test_name)
try:
    os.mkdir(results_folder)
except:
    print(f'Folder {results_folder} already existed.')
runs_folder = os.path.join(PATH_RUNS, test_name)
try:
    os.mkdir(runs_folder)
except:
    print(f'Folder {runs_folder} already existed.')


Proofs Folder Path: 2024_10_31__13_0


Lastly, the existence of a GPU to accelerate the calculations is checked. 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


The existence of a GPU is proofed

## Loading Data: Datasets and Dataloaders

As in the class lab, the data that will be used must be loaded. The very same dataset will be considered, and the same preprocessing will be applied before being sent to the model. 

In [5]:
## Loading Data: Datasets and Dataloaders
train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)

As it was also done in the class lab, the data loading is performed. 

In [6]:
# Sampling method for train/val split
train_proportion = 0.9  
num_train = 50000  

# Generate list with random indexes to choose the examples of each split
indices = list(range(num_train))
split = int(np.floor(train_proportion * num_train))
np.random.shuffle(indices)  # Random rearrange of indices

train_idx, val_idx = indices[:split], indices[split:]
# Generate torch.utils.data.SubsetRandomSampler to get the examples randomly from the given indexes. 
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [7]:
batch_size = 64
num_workers = 0

In [8]:

train_dataset = datasets.CIFAR10(root=PATH_DATA, train=True, 
                                 download=True, transform=train_transform)

val_dataset = datasets.CIFAR10(root=PATH_DATA, train=True, 
                               download=True, transform=val_transform)

test_dataset = datasets.CIFAR10(root=PATH_DATA, train=False,
                                download=True, transform=test_transform)

# DataLoader definition:
# Dataloaders take care of loading data in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          sampler=train_sampler, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                        sampler=val_sampler, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Once batches of examples can be correctly loaded to the training pipeline, it is possible now to proceed with the definition of the model 

## Model Definition

As we did in the class lab, the LeNet-5 Model (basic architecture of CNN) will be considered. This architecture consists of: 
* Convolution Layers: With several filters; each of them produces a map of characteristics. 
* Pooling Layers: Reduces the dimensionality of the input due to an aggregating function. This part is the one that shall be modified by changing the aggregating function. 
* Lineal Layers: Related to a layer in a Neuronal Network. 

![Arquitectura LeNet-5](figures/LeNet-5.png)

## Aggregating Functions Definition

In this section, several aggregating functions will be firstly mathematically explained and secondly programmed so that they could be used in the PoolingLayer. 

In [54]:
p = torch.randn([3,3], dtype=torch.float)
#print(p)
maximum = torch.max(p)
minimum = torch.min(p)
#print(maximum,minimum)
p2 = (p-minimum)/(maximum-minimum)
print(p2)
#p3 = minimum + (maximum-minimum) * p2
#print(p3)
print(OWA(p2, -1, False))

tensor([[0.0000, 1.0000, 0.3429],
        [0.4739, 0.5834, 0.4028],
        [0.6874, 0.5122, 0.6949]])
tensor([0.4606, 0.4898, 0.6436], grad_fn=<SumBackward1>)


### Arithmetic Mean

In [17]:
def arithmetic_mean(X, dim, keepdim):
    return torch.mean(X, dim, keepdim)

### OWA with Learnable Weights

In [53]:
def OWA(X, dim, keepdim):
    
    weight = nn.Parameter(torch.empty(1, X.shape[dim]))
            
    stdv = 1. / math.sqrt(weight.size(1))
    weight.data.uniform_(-stdv, stdv)
    
    tensor_ordered = torch.sort(X, descending = True)
    weight_norm = torch.nn.functional.softmax(weight, dim = dim)
    output = torch.sum(tensor_ordered[0] * weight_norm, dim = dim, keepdim = keepdim)
    
    return output

In [ ]:
class OWALayer(nn.Module):
    
    def __init__(self, in_features, out_features, bias = True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty((out_features, in_features)))
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)
        # Inicializamos los valores de self.weight y self.bias
        self.init_parameters()

    def init_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        # Calculamos el producto matricial entre input y el vector de pesos pero estilo OWA:
        # f(x) = x_1 * w_1 + ... + x_n * w_n
        tensor_orden = torch.sort(input, descending = True)
        weight_norm = torch.nn.functional.softmax(self.weight)
        output = torch.matmul(tensor_orden[0], weight_norm.t())
        if self.bias is not None:
            # f(x) = x_1 * w_1 + ... + x_n * w_n + bias
            output += self.bias
        return output

## Pooling Layer Definition

In [17]:
class AggPoolingLayer(nn.Module):

    def __init__(self, kernel_size, stride, padding= [0,0,0,0], function, dim = -1, keepdim = False):
        super().__init__()
        
        # Una tupla de 2 elementos con los tamaños [𝑘1,𝑘2] de cada ventana a tratar
        self.kernel_size = kernel_size
        
        # Tupla de 2 elementos que indican el número de elementos (en filas y columnas) que 
        # deben saltarse tras reducir cada ventana, hasta encontrar la siguiente a tratar.
        self.stride = stride
        
        # Tupla de 4 elementos de la forma [𝑝𝑎𝑑_𝑙𝑒𝑓𝑡,𝑝𝑎𝑑_𝑟𝑖𝑔ℎ𝑡,𝑝𝑎𝑑_𝑢𝑝,𝑝𝑎𝑑_𝑑𝑜𝑤𝑛] que indica el 
        # número de nuevas filas o columnas a añadir a la entrada, previo a aplicar la agregación.
        self.padding = padding
        
        # Define function and characteristics
        self.function = function
        self.dim = dim
        self.keepdim = keepdim
    
    def forward(self, X):
        
        # Normalize
        maximum = torch.max(X)
        minimum = torch.min(X)
        X = (X-minimum)/(maximum-minimum)
        
        # Añadir columnas/filas según padding
        X_pad = F.pad(X, pad=self.padding, mode='constant', value=0)
        
        # Vamos extrayendo las ventanas a agregar y colocándolas en filas
        X_aux = X_pad.unfold(2, size=self.kernel_size[0], step=self.stride[0]).unfold(3, size=self.kernel_size[1], step=self.stride[1])
        
        # Ponemos el formato correcto
        X_aux = X_aux.reshape([X_aux.shape[0], X_aux.shape[1], X_aux.shape[2], X_aux.shape[3], X_aux.shape[4] * X_aux.shape[5]]) 
        
        
        
        # Agg Func
        Y_temp = self.function(X_aux, dim = self.dim, keepdim = self.keepdim)
        
        # Denormalize 
        Y = minimum + (maximum-minimum) * Y_temp
        
        return Y

<span style="color:green">**Ejercicio 2**: </span> : Implementa el módulo LeNetModel que deberá recrear la arquitectura LeNet-5. Los parámetros de entrada del modelo serán: 

* conv_filters: Una lista con el número de filtros a aprender en cada capa de convolución. Por defecto [64, 64].
* linear_sizes: Una lista con el número de neuronas de salida de cada capa lineal oculta del clasificador. Por defecto [384, 192].
* num_clases: Entero que indica el número de clases a predecir en nuestro problema. Por defecto 10.

La arquitectura de la red consta de:

* Convolución con `conv_filters[0]` filtros de tamaño [2, 2] y stride [1, 1].
* Pooling con *kernel size* [2, 2] y stride [2, 2].
* Convolución con `conv_filters[1]` filtros de tamaño [2, 2] y stride [1, 1].
* Pooling con *kernel size* [2, 2] y stride [2, 2].
* Capa oculta lineal con `conv_filters[1] * 8 * 8` neuronas de entrada y `linear_sizes[0]` neuronas de salida.
* Capa oculta lineal con `linear_sizes[0]` neuronas de entrada y `linear_sizes[1]` neuronas de salida.
* Capa de salida lineal con `linear_sizes[1]` neuronas de entrada y `num_classes` neuronas de salida.

El modelo deberá emplear la función ReLU como función de activación. Además, antes de la primera capa oculta lineal, se deberá convertir la salida $X$ de la capa anterior de tamaño `[batch_size, conv_filters[1], 8, 8]` en un tensor de tamaño `[batch_size, conv_filters[1] * 8, 8]` apto para servir de entrada a una capa *torch.nn.Linear*.

In [47]:
class LeNetModel(nn.Module):

    def __init__(self, conv_filters=[64, 64], linear_sizes=[384, 192], num_classes=10):
        super().__init__()
        self.conv_filters = conv_filters
        
        self.linear_sizes = linear_sizes
        
        self.num_classes = num_classes
        
        # Primera Convolución con conv_filters[0] filtros de tamaño [3, 3] y stride [1, 1] y padding [1, 1].    
        self.conv_1 = torch.nn.Conv2d(in_channels = 3, out_channels = self.conv_filters[0], kernel_size = [3, 3], stride = [1, 1], padding = [1, 1], device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Primer Pooling con kernel size [2, 2] y stride [2, 2]
        self.pool_1 = PoolingLayer(kernel_size = [2, 2], stride = [2, 2])
        
        # Segundo Convolución con conv_filters[1] filtros de tamaño [3, 3] y stride [1, 1] y padding [1, 1].
        self.conv_2 = torch.nn.Conv2d(in_channels = self.conv_filters[0], out_channels = self.conv_filters[1], kernel_size = [3, 3], stride = [1, 1], padding = [1, 1], device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Segundo Pooling con kernel size [2, 2] y stride [2, 2]
        self.pool_2 = PoolingLayer(kernel_size = [2, 2], stride = [2, 2])
        
        # Primera Capa oculta lineal con conv_filters[1] * 8 * 8 neuronas de entrada y linear_sizes[0] neuronas de salida.
        self.lin_1 = torch.nn.Linear(in_features = self.conv_filters[1] * 8 * 8, out_features = self.linear_sizes[0], bias=True, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Segunda Capa oculta lineal con linear_sizes[0] neuronas de entrada y linear_sizes[1] neuronas de salida.
        self.lin_2 = torch.nn.Linear(in_features = self.linear_sizes[0], out_features = self.linear_sizes[1], bias=True, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Capa de salida lineal con linear_sizes[1] neuronas de entrada y num_classes neuronas de salida.
        self.salida = torch.nn.Linear(in_features = self.linear_sizes[1], out_features = self.num_classes, bias=True, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

        
    def forward(self, X):
        
        # print(X.shape)
        
        # Ejecutamos primera Convolución 
        X_tras_conv_1 = self.conv_1(X)
        #print("X_tras_conv_1.shape = ", X_tras_conv_1.shape)
        
        # Ejecutamos primer Pooling 
        X_tras_pool_1 = self.pool_1(X_tras_conv_1)
        #print("X_tras_pool_1.shape = ", X_tras_pool_1.shape)
        
        # Ejecutamos segunda Convolución
        X_tras_conv_2 = self.conv_2(X_tras_pool_1)
        #print("X_tras_conv_2.shape = ", X_tras_conv_2.shape)
        
        # Ejecutamos Segundo Pooling
        X_tras_pool_2 = self.pool_2(X_tras_conv_2)
        #print("X_tras_pool_2.shape = ", X_tras_pool_2.shape)
        
        # Antes de la primera capa oculta lineal, se deberá convertir la salida 𝑋 de la capa anterior de tamaño 
        # [batch_size, conv_filters[1], 8, 8] en un tensor de tamaño [batch_size, conv_filters[1] * 8 * 8]
        X_tras_rshp = X_tras_pool_2.reshape([X_tras_pool_2.shape[0], X_tras_pool_2.shape[1] * X_tras_pool_2.shape[2] * X_tras_pool_2.shape[3]])
        #print("X_tras_rshp.shape = ",X_tras_rshp.shape)
        
        # Ejecutamos 1º Capa oculta lineal.
        X_tras_lin_1 = self.lin_1(X_tras_rshp)
        #print("X_tras_lin_1.shape = ", X_tras_lin_1.shape)
        
        # Le aplicamos ReLU
        X_tras_relu_1 = F.relu(X_tras_lin_1)
        #print("X_tras_relu_1.shape = ",X_tras_relu_1.shape)
        
        # Ejecutamos 2º capa oculta lineal 
        X_tras_lin_2 = self.lin_2(X_tras_relu_1)
        #print("X_tras_lin_2.shape = ", X_tras_lin_2.shape)
        
        # Le aplicamos ReLU
        X_tras_relu_2 = F.relu(X_tras_lin_2)
        #print("X_tras_relu_2.shape = ", X_tras_relu_2.shape)
        
        # Ejecutamos Capa de salida lineal .
        X_salida = self.salida(X_tras_relu_2)
        #print("X_salida.shape = ", X_salida.shape)
        
        return X_salida

## Tensorboard

Antes de pasar a plantear el bucle de entrenamiento de nuestro nuevo modelo, vamos a presentar el modo de loggear información de nuestro modelo de modo que sea compatible con la herramienta [Tensorboard](https://www.tensorflow.org/tensorboard?hl=es-419). Aunque inicialmente se trataba de una herramienta desarrollada para trabajar con el framework Tensorflow (otra alternativa a PyTorch para Deep Learning), actualmente [es compatible con PyTorch](https://pytorch.org/docs/stable/tensorboard.html).

Tensorboard es una herramienta de visualización muy potente que permite, entre otras:

* Registrar la evaluación de métricas de aprendizaje (como coste o tasa de acierto): Permite medir fácilmente el rendimiento del modelo en entrenamiento y compararlo con el de otras variantes del mismo u otros modelos.
* Registrar distribuciones de valores (como las de los parámetros del modelo o sus gradientes): Facilita localizar problemas de entrenamiento del modelo, como "gradientes desvanecientes" o "gradientes explosivos".
* Proyectar datos transformados a un espacio dimensional mejor (mediante algoritmos como PCA o T-SNE): Se puede utilizar sobre los vectores de características extraídos por el modelo a distintos niveles para comprender cómo diferencia el modelo entre las distintas clases. Veremos su uso en futuras prácticas.

Vamos a comenzar ilustrando este proceso con un proceso muy simple, antes de incorporarlo a nuestra pipeline de entrenamiento.

En PyTorch, la forma más fácil de registrar información para su posterior análisis con la herramienta, es a través del objeto *torch.utils.tensorboard.SummaryWriter*. Los pasos a seguir son:

* Crear un *SummaryWriter* indicándole la ruta en la que almacenar los logs:

    `writer = SummaryWriter(log_dir=runs_folder)`

* Para registrar un valor escalar (por ejemplo el coste de nuestra función o la tasa de acierto), utilizaremos:

    `writer.add_scalar(tag, scalar_value, global_step)`
    * *tag* será un *string* con el que se identificará la variable a registrar.
    * *scalar_value* será un *float* que contendrá el valor de esa variable en la iteración que estamos registrando.
    * *global_step* será un *int* que indica la iteración que estamos registrando.

Para ilustrar este proceso vamos a plantear el ejemplo que vimos en el Tutorial para optimizar los parámetros de la función:

$$f(a) = w_3 * (w_1 * a) + w_4 * (w_2 * a)$$

En esta ocasión, no obstante, vamos a utilizar algunos de los conceptos que ya conocemos:

In [23]:
# Tensorboard: Inicializamos nuestro objeto SummaryWriter con la ruta del test actual:
writer = SummaryWriter(log_dir=runs_folder)

# Inicializamos las variables a tratar:
a = torch.tensor([3], dtype=torch.float).to(device)
# w = torch.tensor([-.2, .2, -.4, .3], dtype=torch.float, device=device, requires_grad=True)
w = nn.Parameter(torch.tensor([-2, 2, -4, 3], dtype=torch.float, device=device))
# Nota: Creamos el tensor directamente en "device"
# w = torch.tensor(...).to(device) no nos permitiría fijar w1 como parámetro a optimizar, 
# dado que la operación .to() hace que deje de ser un nodo hoja del grafo de derivación.
# (No tenemos este problema cuando enviamos los parámetros de un modelo mediante model.to(device) 
# como hicimos en la práctica anterior)

# Creamos un Optimizer que utilice el algoritmo SGD:
optimizer = optim.SGD([w,], lr=0.001, weight_decay=0.001)

for i in range(100):
    b = a * w[0]
    c = a * w[1]
    d = w[2] * b + w[3] * c
    l = torch.abs(10 - d)  # Cálculo del coste
    print(f'l={l}')
    # Tensorboard: Registramos el valor de coste en la iteración i
    writer.add_scalar('loss', l.item(), global_step=i)
    l.backward()
    # Tensorboard: Registramos el valor de los gradientes de w:
    print(f'w={w.cpu()}; w.grad={w.grad}')
    writer.add_histogram('w1_grad', w.grad.cpu(), global_step=i)
    writer.add_histogram('w', w.cpu(), global_step=i)
    optimizer.step()

l=tensor([32.], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-2.,  2., -4.,  3.], grad_fn=<ToCopyBackward0>); w.grad=tensor([-12.,   9.,  -6.,   6.], device='cuda:0')
l=tensor([31.7033], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-1.9880,  1.9910, -3.9940,  2.9940], grad_fn=<ToCopyBackward0>); w.grad=tensor([-23.9820,  17.9820, -11.9640,  11.9730], device='cuda:0')
l=tensor([31.1130], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-1.9640,  1.9730, -3.9820,  2.9820], grad_fn=<ToCopyBackward0>); w.grad=tensor([-35.9281,  26.9281, -17.8560,  17.8920], device='cuda:0')
l=tensor([30.2351], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-1.9281,  1.9461, -3.9642,  2.9641], grad_fn=<ToCopyBackward0>); w.grad=tensor([-47.8206,  35.8204, -23.6403,  23.7303], device='cuda:0')
l=tensor([29.0784], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-1.8803,  1.9103, -3.9405,  2.9404], grad_fn=<ToCopyBackward0>); w.grad=tensor([-59.6421,  44.6416, -29.2811,  29.4611], device='

w=tensor([ 1.4654, -0.7721, -1.2287,  0.5749], grad_fn=<ToCopyBackward0>); w.grad=tensor([  -9.6623,   28.3586, -131.5021,   89.2785], device='cuda:0')
l=tensor([16.0212], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([ 1.4750, -0.8005, -1.0972,  0.4856], grad_fn=<ToCopyBackward0>); w.grad=tensor([  -6.3708,   26.9018, -135.9273,   91.6800], device='cuda:0')
l=tensor([15.2497], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([ 1.4814, -0.8274, -0.9612,  0.3939], grad_fn=<ToCopyBackward0>); w.grad=tensor([  -3.4871,   25.7200, -140.3715,   94.1622], device='cuda:0')
l=tensor([14.4238], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([ 1.4849, -0.8531, -0.8209,  0.2998], grad_fn=<ToCopyBackward0>); w.grad=tensor([  -1.0246,   24.8208, -144.8262,   96.7216], device='cuda:0')
l=tensor([13.5483], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([ 1.4859, -0.8779, -0.6760,  0.2030], grad_fn=<ToCopyBackward0>); w.grad=tensor([   1.0035,   24.2117, -149.2840,   99.3554], device='cuda:

w=tensor([-0.6434, -0.6264,  6.3319, -4.1141], grad_fn=<ToCopyBackward0>); w.grad=tensor([ -91.1672,   77.9524, -103.0457,   83.8917], device='cuda:0')
l=tensor([11.7909], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-0.5522, -0.7043,  6.4350, -4.1979], grad_fn=<ToCopyBackward0>); w.grad=tensor([-110.4721,   90.5463, -101.3890,   86.0047], device='cuda:0')
l=tensor([8.4472], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-0.4418, -0.7949,  6.5364, -4.2839], grad_fn=<ToCopyBackward0>); w.grad=tensor([-130.0811,  103.3981, -100.0636,   88.3893], device='cuda:0')
l=tensor([4.4230], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-0.3117, -0.8983,  6.6364, -4.3723], grad_fn=<ToCopyBackward0>); w.grad=tensor([-149.9904,  116.5151,  -99.1286,   91.0842], device='cuda:0')
l=tensor([0.3207], device='cuda:0', grad_fn=<AbsBackward0>)
w=tensor([-0.1617, -1.0148,  6.7355, -4.4634], grad_fn=<ToCopyBackward0>); w.grad=tensor([-129.7838,  103.1248,  -99.6137,   88.0398], device='cuda:0')

Con esto, en la carpeta *runs_folder* se habrá creado un fichero con nombre "events.out.tfevents.XXX", con la parte final del nombre correspondiendo a un identificador del modelo.

Para analizar este log a continuación a través de la herramienta Tensorboard, deberemos:

* Desde la terminal de linux, situarnos en la carpeta PATH_RUNS
* Una vez en esta carpeta, escribimos el siguiente comando `tensorboard --logdir nombre_test`, que inicializará Tensorboard y analizará los *logs* almacenados dentro de la carpeta nombre_test (corresponderá a la ruta de *runs_folder*)
* Abrimos una ventana de cualquier navegador y accedemos a la URL *localhost:6006* (o el número del puerto que nos indique la terminal).

## Bucle de entrenamiento

En esta ocasión reutilizaremos el bucle de la práctica anterior, con unas leves modificaciones.

<span style="color:green">**Ejercicio 3**: </span> : Añade a la función *train* presentada a continuación, las líneas de código que sean necesarias para que, mediante Tensorboard, registres la siguiente información:

* Valor de la función de coste tras cada epoch en Train y Val
* Valor de la tasa de acierto tras cada epoch en Train y Val
* Valor de los parámetros de cada capa al terminar cada epoch
* Valor de los gradientes de los parámetros de cada capa al terminar cada epoch

**NOTA**: En la práctica anterior <span style="color:red">había un error</span> en la definición de train(), dado que no se le enviaban el *criterion* a utilizar para calcular el coste de nuestro modelo ni el *optimizer* encargado de actualizar los valores de los parámetros.

In [48]:
def train(model, train_loader, criterion, optimizer, val_loader=None, num_epochs=20, device='cuda'):

    # Listas para generar logs durante el entrenamiento:
    train_acc = []  
    train_loss = []
    
    # Escritor 
    writer = SummaryWriter(log_dir=runs_folder)
    
    if val_loader is not None:
        val_acc = []
        val_loss = []

    # Bucle de entrenamiento:
    for epoch in range(num_epochs):
        
        running_loss = 0.0  # Acumulamos el valor de coste obtenido tras cada epoch
        
        count_evaluated = 0
        
        count_correct = 0
        
        for batch_idx, data in enumerate(train_loader, 0):  
            
            model.train()  
            
            inputs, labels = data[0].to(device), data[1].to(device)  
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            ### Fase de log ###
            running_loss += loss.item()  # Acumulamos el error obtenido para utilizarlo
                # a la hora de generar logs del proceso.
                
            # Contamos el número de ejemplos evaluados y acertados:
            
            count_evaluated += inputs.shape[0]
            
            count_correct += torch.sum(labels == torch.max(outputs, dim=1)[1])
            
        # Log del valor de la función de coste y accuracy
        print('Training: [%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / (batch_idx+1)))
        
        # Apuntamos valor función coste 
        writer.add_scalar('Valor Coste Train', running_loss / (batch_idx+1), global_step = epoch + 1)
        
        train_loss.append(running_loss / (batch_idx+1))
        # Almacenamos la accuracy al final de la epoch (en train)
        
        train_acc.append(float(count_correct) / count_evaluated)
        
        # Apuntamos valor función coste 
        writer.add_scalar('Valor Accuracy Train', float(count_correct) / count_evaluated, global_step = epoch + 1)
        
        # Apuntamos Log de los gradientes de cada parámetro
        for name, param in model.named_parameters():
            if param.grad is not None:
                writer.add_histogram(f'{name}.grad', param.grad, epoch + 1)
        
        ### Fase de validación ### 
        if val_loader is not None:
            running_loss_val = 0.0
            count_evaluated = 0
            count_correct = 0
            model.eval()
            with torch.no_grad():
                for val_batch_idx, data_val in enumerate(val_loader, 0):
                    inputs_val, labels_val = data_val[0].to(device), data_val[1].to(device)
                    outputs_val = model(inputs_val)
                    loss = criterion(outputs_val, labels_val)
                    running_loss_val += loss.item()
                    count_evaluated += inputs_val.shape[0]
                    count_correct += torch.sum(labels_val == torch.max(outputs_val, dim=1)[1])
                # Presentamos el resumen de la validación de la epoch:
                val_loss.append(running_loss_val / (val_batch_idx + 1))
                
                # Apuntamos valor función coste 
                writer.add_scalar('Valor Coste Val', running_loss_val / (val_batch_idx + 1), global_step = epoch + 1)
                
                acc_val = float(count_correct) / count_evaluated
                
                # Apuntamos valor función coste 
                writer.add_scalar('Valor Accuracy Val', float(count_correct) / count_evaluated, global_step = epoch + 1)
                
                print('Validation: epoch %d - acc: %.3f' %
                            (epoch + 1, acc_val))
                val_acc.append(acc_val)
                
        
        # Apuntamos valor de los parámetros al finalizar la iteración
        writer.add_scalar('Running Loss', running_loss, global_step = epoch + 1)
        writer.add_scalar('Count Evaluated', count_evaluated, global_step = epoch + 1)
        writer.add_scalar('Count Correct', count_correct, global_step = epoch + 1)
        
    # Devolvemos, tanto el modelo entrenado, como el diccionario con las estadísticas del entrenamiento
    return model, 

Y aquí definimos la función de evaluación. 

**NOTA**: En la práctica anterior <span style="color:red">había un error</span> en la definición de test(), dado que no se le enviaban el *criterion* a utilizar para calcular el coste de nuestro modelo.

In [49]:
def test(model, test_loader, criterion, device='cuda'):
    with torch.no_grad():
        number_samples = 0
        number_correct = 0
        running_loss_test = 0.0
        for test_batch_idx, data_test in enumerate(test_loader, 0):
            inputs_test, labels_test = data_test[0].to(device), data_test[1].long().to(device)
            outputs_test = model(inputs_test)
            loss = criterion(outputs_test, labels_test)
            running_loss_test += loss.cpu().numpy()
            # Accuracy:
            _, outputs_class = torch.max(outputs_test, dim=1)
            number_correct += torch.sum(outputs_class == labels_test).cpu().numpy()
            number_samples += len(labels_test)
        acc_test = number_correct / number_samples
        print('Test - Accuracy: %.3f' % acc_test)
        print('Test - CrossEntropy: %.3f' % (running_loss_test / (test_batch_idx+1)))

### Entrenamiento del modelo:

Como en la práctica anterior, debemos fijar los hiperparámetros a utilizar para nuestro modelo, y proceder con su entrenamiento y evaluación. La parte final de esta práctica será recrear este proceso.

<span style="color:green">**Ejercicio 4**: </span> : Prepara los siguientes puntos:
* Modelo LeNetPlus con los siguientes parámetros:
    * Dos capas convolucionales con 64 filtros cada una
    * Dos capas lineales ocultas con 384 y 192 neuronas respectivamente
* Función de coste: Cross Entropy
* Optimizador SGD con los siguientes hiperparámetros:
    * learning_rate = 0.001
    * momentum = 0.9
    * weight_decay = 0.0001

A continuación, entrena el modelo y evalúalo con el conjunto de test.

Por último, analiza la evolución del entrenamiento utilizando la herramienta Tensorboard, en base a los valores registrados.

In [50]:
# Creación del modelo
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LeNetModel(conv_filters=[64, 64], linear_sizes=[384, 192], num_classes=10).to(device)
print(model)

LeNetModel(
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_1): PoolingLayer()
  (conv_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_2): PoolingLayer()
  (lin_1): Linear(in_features=4096, out_features=384, bias=True)
  (lin_2): Linear(in_features=384, out_features=192, bias=True)
  (salida): Linear(in_features=192, out_features=10, bias=True)
)


In [51]:
# Definición de función de coste y optimizador
learning_rate = 0.001
momentum = 0.9
weight_decay = 0.0001

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


In [52]:
# Entrenamiento
model = train(model, train_loader, criterion, optimizer, val_loader, num_epochs= 20, device=device)

Training: [1,   704] loss: 2.016
Validation: epoch 1 - acc: 0.371
Training: [2,   704] loss: 1.657
Validation: epoch 2 - acc: 0.425
Training: [3,   704] loss: 1.495
Validation: epoch 3 - acc: 0.486
Training: [4,   704] loss: 1.386
Validation: epoch 4 - acc: 0.517
Training: [5,   704] loss: 1.308
Validation: epoch 5 - acc: 0.543
Training: [6,   704] loss: 1.247
Validation: epoch 6 - acc: 0.562
Training: [7,   704] loss: 1.192
Validation: epoch 7 - acc: 0.583
Training: [8,   704] loss: 1.144
Validation: epoch 8 - acc: 0.595
Training: [9,   704] loss: 1.103
Validation: epoch 9 - acc: 0.608
Training: [10,   704] loss: 1.062
Validation: epoch 10 - acc: 0.617
Training: [11,   704] loss: 1.019
Validation: epoch 11 - acc: 0.627
Training: [12,   704] loss: 0.986
Validation: epoch 12 - acc: 0.638
Training: [13,   704] loss: 0.951
Validation: epoch 13 - acc: 0.647
Training: [14,   704] loss: 0.925
Validation: epoch 14 - acc: 0.661
Training: [15,   704] loss: 0.893
Validation: epoch 15 - acc: 0.66

TypeError: cannot unpack non-iterable LeNetModel object

In [ ]:
# Graficar cosas
train_loss = train_stats['train_loss']
train_acc = train_stats['train_acc']
val_loss = train_stats['val_loss']
val_acc = train_stats['val_acc']

# Error en entrenamiento:
plt.figure()
plt.plot(range(len(train_loss)), train_loss)
plt.xlabel('Iteraciones')
plt.ylabel('Loss')
plt.title('Training loss')
plt.show()
# Error en validación:
plt.figure()
plt.plot(range(len(val_loss)), val_loss)
plt.xlabel('Iteraciones')
plt.ylabel('Loss')
plt.title('Validation loss')
plt.show()
# Accuracy en entrenamiento y validación:
plt.figure()
plt.plot(range(len(train_acc)), train_acc, 'r--', range(len(val_acc)), val_acc, 'b')
plt.xlabel('Iteraciones')
plt.ylabel('Accuracy')
plt.legend(['Train set', 'Validation set'])
plt.title('Model accuracy')
plt.show()

In [ ]:
# Test
test(model, test_loader, criterion = criterion, device = device)